In [1]:
import pandas as pd

In [46]:
from collections import Counter

- [ctr data from Kaggle competition](https://www.kaggle.com/c/avazu-ctr-prediction/data)
- train_subset is first 10K rows of 6+GB set

In [68]:
df = pd.read_csv('data/train_subset.csv')

In [69]:
df.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,1.000009e+18,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,2,15706,320,50,1722,0,35,-1,79
1,1.000017e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
2,1.000037e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
3,1.000064e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15706,320,50,1722,0,35,100084,79
4,1.000068e+19,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,...,1,0,18993,320,50,2161,0,35,-1,157


In [284]:
# how many features should we have after?
len(df['device_id'].unique())

7201

Features are $\theta$ = [$N^+$, $N^-$, $log(N^+)-log(N^-)$, isRest]

$N^+$ = $p(+)$ = $n^+/(n^+ + n^-)$

$N^-$ = $p(-)$ = $n^-/(n^+ + n^-)$

$log(N^+)-log(N^-)$ = $\frac{p(+)}{p(-)}$

isRest = back-off bin (not shown here)

In [267]:
def click_counting(x):
    clicks = pd.Series(x[x['click'] > 0]['device_id'].value_counts(), name='clicks')
    no_clicks = pd.Series(x[x['click'] < 1]['device_id'].value_counts(), name='no_clicks')
    
    counts = pd.DataFrame([clicks,no_clicks]).T.fillna('0')
    counts['total_clicks'] = counts['clicks'].astype('int64') + counts['no_clicks'].astype('int64')
    
    return counts

def bin_counting(counts):
    counts['N+'] = counts['clicks'].astype('int64').divide(counts['total_clicks'].astype('int64'))
    counts['N-'] = counts['no_clicks'].astype('int64').divide(counts['total_clicks'].astype('int64'))
    counts['log_N+'] = counts['N+'].divide(counts['N-'])

#    If we wanted to only return bin-counting properties, we would filter here
    bin_counts = counts.filter(items= ['N+', 'N-', 'log_N+'])
    return counts, bin_counts

In [271]:
# bin counts example: device_id
device_clicks = click_counting(df.filter(items= ['device_id', 'click']))
device_all, device_bin_counts = bin_counting(device_clicks)

In [281]:
# check to make sure we have all the devices
len(device_bin_counts)

7201

In [272]:
device_all.sort_values(by = 'total_clicks', ascending=False).head(10)

,clicks,no_clicks,total_clicks,N+,N-,log_N+
a99f214a,15729,71206,86935,0.180928,0.819072,0.220894
c357dbff,33,134,167,0.197605,0.802395,0.246269
31da1bd0,0,62,62,0.000000,1.000000,0.000000
936e92fb,5,54,59,0.084746,0.915254,0.092593
a167aa83,0,55,55,0.000000,1.000000,0.000000
971d35f0,1,46,47,0.021277,0.978723,0.021739
3c0208dc,0,47,47,0.000000,1.000000,0.000000
d2bbb640,0,46,46,0.000000,1.000000,0.000000
e8440dbf,0,46,46,0.000000,1.000000,0.000000
fc51ca94,0,44,44,0.000000,1.000000,0.000000


In [278]:
# We can see how this can change model evaluation time by comparing raw vs. bin-counting size
from sys import getsizeof

print('Our pandas Series, in bytes: ', getsizeof(df.filter(items= ['device_id', 'click'])))
print('Our feature, in bytes: ', getsizeof(device_bin_counts))

Our pandas Series, in bytes:  7300031
Our feature, in bytes:  640913
